# Imports

In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

In [2]:
import pandas as pd
from google.cloud import bigquery
import plaid
from utils.bq_utils import BqUtils
from utils.plaid_utils import PlaidUtils
from sql.bq_table_schemas import BqTableSchemas

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)


PLAID_CLIENT_ID = "65975384ab670e001c0aaf0d"
# PLAID_SECRET="56e33c77237c8c9e45f5c066b8b2fa" #production
PLAID_SECRET = "9294dd5ca4a5c99d90da56640f40e5"  # sandbox
PLAID_ENV = "sandbox"
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
PLAID_HOST = plaid.Environment.Sandbox
# PLAID_REDIRECT_URI="https://localhost:3000/"
PLAID_ACCESS_TOKENS = [
    "access-sandbox-1902b6d6-33f3-49b8-a3ce-3213a5fe9a49",
    "access-sandbox-12481aea-33b6-4101-9f49-4e07ebbc1431",
    "access-sandbox-f415798a-74af-4e7f-b504-dc075dc5889c",
]


bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_init = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
bq_tables = BqTableSchemas()

plaid_cursors_bq = bq_tables.plaid_cursors_YYYYMMDD()

# Test Transactions Sync

In [ ]:
plaid_init.create_temp_cursors_bq_table(confirm=False)

latest_cursors_df = plaid_init.get_latest_cursors()

# print(latest_cursors_df["access_token"][0])
# transactions_df, removed_df = plaid_client.get_transactions(
#     access_token=latest_cursors_df["access_token"][0],
#     item_id=latest_cursors_df["item_id"][0],
#     next_cursor=latest_cursors_df["next_cursor"][0],
# )

In [ ]:
from plaid.model.transactions_sync_request import TransactionsSyncRequest

In [ ]:
transactions_df.groupby(["personal_finance_category_primary", "personal_finance_category_detailed"], as_index=False)[
    "transaction_id"
].nunique()

In [ ]:
# transactions_df["personal_detailed"] = transactions_df.apply(
#     lambda row: row["personal_finance_category_detailed"][len(row["personal_finance_category_primary"]) + 1 :], axis=1
# )

# transactions_df.groupby(
#     ["personal_finance_category_primary", "personal_finance_category_detailed", "personal_detailed"], as_index=False
# )["transaction_id"].nunique()

# Test Delete all Partitions

In [8]:
removed_transaction_bq = bq_tables.plaid_removed_transactions_YYYYMMDD()
transaction_bq = bq_tables.plaid_transactions_YYYYMMDD()
accounts_bq = bq_tables.plaid_accounts()

In [3]:
bq.concat_table_name("project", "data", "table")

'project.data.table'

In [10]:
bq.delete_all_partitions(
    project_id=transaction_bq["project_id"],
    dataset_id=transaction_bq["dataset_id"],
    table_id=transaction_bq["table_id"],
    confirm=True,
)


SUCCESS: `zsc-personal.personal_finance.plaid_transactions_20240413` successfully deleted!


In [4]:
from bqt import bqt